In [43]:
import astroprov
from astropy.io import fits
from astropy.io.ascii import SExtractor
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import os
from photutils import DAOStarFinder
from photutils import CircularAperture
from photutils import aperture_photometry
#import subproc

import numpy as np
from astropy import wcs
from astropy.io import fits
import sys

from astroquery.simbad import Simbad

import matplotlib.pyplot as plt
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table
from astropy.table import Column

import collections
import subprocess

import time
from multiprocessing import Pool

In [2]:
imagedirectory = '/home/mj1e16/keplerImages'

newImagelist = []
Image = os.listdir(imagedirectory)
for i, item in enumerate(Image):
    if item.find('.fits') != -1:
        #print(i,Image[i].find('.fits'))
        newImagelist.append(item)
Image = newImagelist


FWHM = 5.0
MinSep = 0.8 #sets value for discriminating sources
N = len(Image) #number of images minus 1 as it starts from zero
imno = 0
imagedata = []
sources = dict()
Date = []
STD = []
Median = []


os.chdir(imagedirectory)
def daoStarFinder(imagename):
    """
    Locates and measures the brightness of all objects in the image as well as the sky background
    and other image data.

    :param imagenumber: Position of image in directory
    :return: Table of object data, date of observation, image data for backround calculation
    """
    #print(Image[imagenumber])
    hdu_list = fits.open(imagename) # open image header
    imagedata = (hdu_list[0].data) 
    mean, median, std = sigma_clipped_stats(imagedata, sigma=3.0, iters=5) 
    Median.append(median)
    STD.append(std)
    daofind = DAOStarFinder(fwhm=FWHM, threshold=5.*std) # iraf star finder 
    Sources = daofind(imagedata - median)
    #Date = (hdu_list[0].header['MJD'])
    output = {}
    output[0] = Sources
    output[1] = Date
    output[2] = imagedata
    
#     fig, ax = plt.subplots(figsize=(10, 10))
#     positions = (output[0]['xcentroid'], output[0]['ycentroid'])
#     apertures = CircularAperture(positions, r=4.)
#     norm = ImageNormalize(stretch=SqrtStretch())
#     ax.imshow(imagedata, cmap='Greys', origin='lower', vmin=-100,vmax=100)#norm=norm)
#     apertures.plot(color='blue', lw=1.5, alpha=0.5)
#     plt.savefig('fits.png',dpi=100)
#     plt.show()
    
    
    
#     os.chdir(bindingsdirectory)
#     astroprov.provcall([imagenumber],[output[0],output[1],output[2]],"daoStarFinder_PythonCode2Images_SQ_tmpl.provn","daoStarFinder")
#     os.chdir(imagedirectory)
    return output




In [3]:
def load_wcs_from_file(filename,pixCoord):
    # Load the FITS hdulist using astropy.io.fits
    hdulist = fits.open(filename)

    # Parse the WCS keywords in the primary HDU
    w = wcs.WCS(hdulist[0].header)

    # Print out the "name" of the WCS, as defined in the FITS header
    #print(w.wcs.name)

    # Print out all of the settings that were parsed from the header
    w.wcs.print_contents()

    # Three pixel coordinates of interest.
    # Note we've silently assumed a NAXIS=2 image here.
    # Note also that the pixel coordinates are pairs of [X, Y], 
    # and since WCS built from FITS header automatically has
    # origin set to 1, [0, 0] is actually not inside the image.
    pixcrd = np.array(pixCoord, dtype=np.float_)

    # Convert pixel coordinates to world coordinates
    # The second argument is "origin" -- in this case we're declaring we
    # have 1-based (Fortran-like) coordinates.
    world = w.wcs_pix2world(pixcrd, 1)
    #print(world)
    return(world)
#     # Convert the same coordinates back to pixel coordinates.
#     pixcrd2 = w.wcs_world2pix(world, 1)
#     print(pixcrd2)

#     # These should be the same as the original pixel coordinates, modulo
#     # some floating-point error.
#     assert np.max(np.abs(pixcrd - pixcrd2)) < 1e-6




In [4]:
def queryObjectRegion(outputTable,imageName,inCat):
    imageCoords = []
    r = 1*u.arcminute
    
    if inCat == 'dao':
        xlabel = 'xcentroid'
        ylabel = 'ycentroid'
        label = 'id'
        
    if inCat == 'sex':
        xlabel = 'X_IMAGE'
        ylabel = 'Y_IMAGE'
        label = 'NUMBER'
        
    tableLength = outputTable[label][-1]
    for x in range(tableLength):
        xpos = outputTable[xlabel][x]
        ypos = outputTable[ylabel][x]
        if 20 < ypos < 1040:
            if 12 < xpos < 1110:
                imageCoords.append([xpos,ypos])
                # this may need to change per image but I hope not

    coords = load_wcs_from_file(imageName,imageCoords)
    customSimbad = Simbad()
    customSimbad.add_votable_fields('ra(d)','dec(d)','otype')
    customSimbad.remove_votable_fields('coordinates')
            
    result_table = Table([['-'],[0],[0],['-'],[0],[0]],names=('MAIN_ID','RA_d','DEC_d','OTYPE','xcentroid','ycentroid'),dtype=('object','float64','float64','object','float64','float64'))

    for x in range(len(imageCoords)):
        try:
            c = SkyCoord(coords[x][0],coords[x][1],frame='icrs',unit='deg')
            rt = customSimbad.query_region(c,radius=r)
            rowlist = []
            for y in range(len(rt[0])):
                rowlist.append(rt[0][y])
            rowlist.append(imageCoords[x][0])
            rowlist.append(imageCoords[x][1])
            result_table.add_row(rowlist)

        except:
            result_table.add_row(['N/A',[0],[0],'Possible Transient',[imageCoords[x][0]],[imageCoords[x][1]]])
    
#     hdu_list = fits.open(imageName)
#     imagedata = (hdu_list[0].data) 
#     fig, ax = plt.subplots(figsize=(10, 10))
#     positions = (outputTable[xlabel], outputTable[ylabel])
#     apertures = CircularAperture(positions, r=4.)
#     norm = ImageNormalize(stretch=SqrtStretch())
#     ax.imshow(imagedata, cmap='Greys', origin='lower', vmin=-100,vmax=100)#norm=norm)
#     apertures.plot(color='blue', lw=1.5, alpha=0.5)
#     #plt.savefig('fits.png',dpi=100)
#     plt.show()
    
    return result_table

In [62]:
# def alterDefault(defaultDir,valList,attributeList):
    
#     with open(defaultDir+'/default.sex','r') as f:
#         data  = f.read()
        
#     catName = '/data/mj1e16/sexOutTables/table'
#     for x in range(len(valList)):
#         nameLoc = data.find(attributeList[x]) + len(attributeList[x])
#         endLoc = data[nameLoc:].find('#') + nameLoc
#         newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
#         data = newData
#         catName += '_'+str(valList[x])
    
#     catName += '.cat'
#     name = 'CATALOG_NAME'
#     nameLoc = data.find(name) + len(name)
#     endLoc = data[nameLoc:].find('#') + nameLoc
#     newData = data[:nameLoc] + ' '+catName+' ' + data[endLoc:]
#     data = newData
#     with open(defaultDir+'/default.sex','w') as f:
#         f.write(data)

In [90]:
def alterDefault(valList,imagename='/home/mj1e16/iraf/editedImage5000.fits',defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',attributeList=['DETECT_THRESH','DETECT_MINAREA','BACK_TYPE','BACK_VALUE','BACK_FILTERSIZE','BACK_SIZE']):
    
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
        
    catName = '/data/mj1e16/sexOutTables/table'
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData
        catName += '_'+str(valList[x])
    
#     catName += '.cat'
#     name = 'CATALOG_NAME'
#     nameLoc = data.find(name) + len(name)
#     endLoc = data[nameLoc:].find('#') + nameLoc
#     newData = data[:nameLoc] + ' '+catName+' ' + data[endLoc:]
#     data = newData
    
    with open(defaultDir+'/default.sex','w') as f:
        f.write(data)
    os.chdir(defaultDir)
    subprocess.call(['sex',imagename])

In [57]:
def sextractor(sextractoryDir,imagename):
    os.chdir(sextractoryDir)
    subprocess.call(['sex',imagename])

In [ ]:
def sexOutput(defaultDir,valList,attributeList,imagename,cataloguename):
    alterDefault(defaultDir,valList,attributeList)
    outTab = sextractor(defaultDir,imagename,cataloguename)
    tableName = '/home/mj1e16/outTablesSex/table'
    for x in range(len(valList)):
        tableName += '_'+str(valList[x])+'_'
    tableName += '.txt'
    outTab.write(tableName,format='ecsv')

In [8]:
def findObjectOccurences(inputTable,columnName):
    otype = inputTable[columnName]
    otypelist = []
    for x in range(len(otype)):
        otypelist.append(otype[x])
    #print(len(otypelist))
    newotype = set(otypelist)
    counter = collections.Counter(otypelist)
    keys = counter.keys()
    values = counter.values()
    occurenceDict = dict(zip(keys,values))
    od = collections.OrderedDict(sorted(occurenceDict.items()))
    return od

In [9]:
def foundInBoth(daoTable,sexTable):
    
    # currently works after both have been checked through vizier - change sex x and y labels to X_IMAGE and Y_IMAGE otherwise
    simdifsize = 5
    
    daox = daoTable['xcentroid'].tolist()
    #daox = [round(x,1) for x in daox]
    daoy = daoTable['ycentroid'].tolist()
    #daoy = [round(x,1) for x in daoy]
    #daocoords = zip(daox,daoy)
    #print(daocoords)
    
    sexx = sexTable['xcentroid'].tolist()
    #sexx = [round(x,1) for x in sexx]
    sexy = sexTable['ycentroid'].tolist()
    #sexy = [round(x,1) for x in sexy]

    commonIndicies = [[x,y] for x,y,z,v in zip(daox,daoy,sexx,sexy) if z-simdifsize < x < z+simdifsize and v-simdifsize < y < v+simdifsize]
    
    return commonIndicies

In [10]:
imageName = 'difkplr2009115080620_ffi-cal57.fits'

In [20]:
t0 = time.time()
sextable = sextractor('/home/mj1e16/sextractor/sextractor-master/config/','difkplr2009115080620_ffi-cal57.fits[0]','test.cat')
print(time.time()-t0)

0.254105806351


In [51]:
sextable.write('/home/mj1e16/testtab.txt',format='ascii.ecsv',overwrite=True)

In [12]:
output = daoStarFinder('difkplr2009115080620_ffi-cal57.fits')
daotable = output[0]
Date = output[1]
imagedata = output[2]

In [13]:
#coords = load_wcs_from_file('difkplr2009115080620_ffi-cal57.fits',imageCoords)

In [14]:
result_table_dao = queryObjectRegion(daotable,'difkplr2009115080620_ffi-cal57.fits','dao')

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '19:34:18.8953 +47:44:41.8572': No astronomical object found :
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '19:32:04.096 +47:34:21.8359': No astronomical object found :
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '19:31:32.8117 +47:31:20.3122': No astronomical object found :
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ast

In [15]:
result_table_sex = queryObjectRegion(sextable,'difkplr2009115080620_ffi-cal57.fits','sex')

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '19:36:08.9234 +48:24:20.7876': No astronomical object found :
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '19:35:07.605 +48:18:11.1485': No astronomical object found :
  (error.line, error.msg))


In [55]:
#commonObjects = foundInBoth(result_table_dao,result_table_sex)
#make final image w/ fake stars, test run, make 6 versions and test multiprocessing - RUN!
dDir = '/home/mj1e16/sextractor/sextractor-master/config/'
imageName = '/home/mj1e16/iraf/editedImage5000.fits'

In [69]:
atList = ['DETECT_THRESH','DETECT_MINAREA','BACK_TYPE','BACK_VALUE','BACK_FILTERSIZE','BACK_SIZE']

valList = [np.linspace(1.5,5,35),np.linspace(1,20,20),['AUTO','MANUAL'],np.linspace(0,20,41),np.linspace(1,11,11),[16,32,64,128,256,512]]

# for x in range(len(atList)):
#     valueList = []
#     for y in range(len(valList[x])):
#         valueList.append(valList[x][y])
p = Pool(6)
for x0 in range(len(valList[0])):
    for x1 in range(len(valList[1])):
        for x2 in range(len(valList[2])):
            for x3 in range(len(valList[3])):
                for x4 in range(len(valList[4])):
                    #for x5 in range(1): #len(valList[5])):
                    valueList = [valList[0][x0],valList[1][x1],valList[2][x2],valList[3][x3],valList[4][x4]]# ,valList[5][0]]
                    valueList.append(0)
                    fullValList = []
                    for x in range(len(valList[5])):
                        valueList[5] = valList[5][x]
                        fullValList.append(newval)
                    p.map(alterDefault,fullValList)
                    #alterDefault(dDir,valueList,atList)
                    #sextractor(dDir,imageName)

TypeError: object of type 'NoneType' has no len()

In [85]:
atList = ['DETECT_THRESH','DETECT_MINAREA','BACK_TYPE','BACK_VALUE','BACK_FILTERSIZE','BACK_SIZE']

valList = [np.linspace(1.5,5,35),np.linspace(1,20,20),['AUTO','MANUAL'],np.linspace(0,20,41),np.linspace(1,11,11),[16,32,64,128,256,512]]

# for x in range(len(atList)):
#     valueList = []
#     for y in range(len(valList[x])):
#         valueList.append(valList[x][y])
p = Pool(6)
for x0 in range(1): #len(valList[0])):
    for x1 in range(1): #len(valList[1])):
        for x2 in range(1): #len(valList[2])):
            for x3 in range(1): #len(valList[3])):
                for x4 in range(1): #len(valList[4])):
                    fullValList = []
                    for x5 in range(1): #len(valList[5])):
                        #valueList = [valList[0][x0],valList[1][x1],valList[2][x2],valList[3][x3],valList[4][x4],valList[5][x5]]# ,valList[5][0]]
                        fullValList.append([valList[0][x0],valList[1][x1],valList[2][x2],valList[3][x3],valList[4][x4],valList[5][x5]])
                    #print(fullValList)
                    p.map(alterDefault,fullValList)

print(valueList)                    
#print()                    
print(fullValList)

# Default configuration file for SExtractor 2.12.4
# EB 2010-10-10
#
 
#-------------------------------- Catalog ------------------------------------
 
CATALOG_NAME /data/mj1e16/sexOutTables/table_1.5_1.0_AUTO_0.0_1.0_16.cat # name of the output catalog
CATALOG_TYPE     ASCII_HEAD     # NONE,ASCII,ASCII_HEAD, ASCII_SKYCAT,
                                # ASCII_VOTABLE, FITS_1.0 or FITS_LDAC
PARAMETERS_NAME  default.param  # name of the file containing catalog contents
 
#------------------------------- Extraction ----------------------------------
 
DETECT_TYPE      CCD            # CCD (linear) or PHOTO (with gamma correction)
DETECT_MINAREA 1.0 # min. # of pixels above threshold

DETECT_THRESH 1.5 # <sigmas> or <threshold>,<ZP> in mag.arcsec-2
ANALYSIS_THRESH  1.5            # <sigmas> or <threshold>,<ZP> in mag.arcsec-2
 
FILTER           Y              # apply filter for detection (Y or N)?
FILTER_NAME      default.conv   # name of the file containing the filter
 
DEBLEND_NTHRESH

In [75]:
a = [0,1,2]
a.extend([3])
print(a)

[0, 1, 2, 3]


In [87]:
ten = 100 
a = 35*20*2*41*11*6
print(a)

3788400


In [89]:
(a*726)/1000000

2750

In [42]:
secs = a*0.25
days = secs/86400
print(days)
years = days/365.25
print(years)

0.972222222222
0.00266179937638


In [16]:
otypes = findObjectOccurences(result_table_sex,'OTYPE')
#otypyes 
print('Sex',otypes)
otypes = findObjectOccurences(result_table_dao,'OTYPE')
print('dao',otypes)

('Sex', OrderedDict([('-', 1), ('Possible Transient', 2482), ('RGB*', 2), ('Radio', 1), ('Star', 8)]))
('dao', OrderedDict([('-', 1), ('Eruptive*', 2), ('Possible Transient', 503), ('RGB*', 4), ('RotV*', 1), ('Star', 19)]))


In [17]:
mask = (result_table_sex['OTYPE']=='Inexistent')
print(result_table_sex[mask])

MAIN_ID RA_d DEC_d OTYPE xcentroid ycentroid
------- ---- ----- ----- --------- ---------
